In [1]:
from IPython.display import display, HTML
display(HTML ("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

In [2]:
import os # 모델 저장할 폴더가 있는 여부와 폴더 생성
import numpy as np
import pandas as pd # get_dummies(원핫인코딩), crosstab
import seaborn as sns # iris 데이터 프레임 가져오기
from sklearn import datasets # iris 데이터(X, y) 가져오기
from sklearn.preprocessing import LabelEncoder # 라벨인코더
from tensorflow.keras.utils import to_categorical # 원핫인코딩
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Input, Dense, Dropout, LeakyReLU
from tensorflow.keras import metrics # compile
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint
import matplotlib.pyplot as plt

# 1. 기본적인 DNN

In [54]:
# 1. 데이터 생성 및 전처리
iris = datasets.load_iris()
iris_X = iris.data
iris_y = iris.target
train_X , test_X, train_y, test_y = train_test_split(iris_X, iris_y,
                                                    test_size=0.2,
                                                    stratify=iris_y)
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [55]:
# 2. 모델구성
model = Sequential([
    Input(4),
    Dense(units=64, activation='relu'),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=50, activation='relu'),
    Dropout(0.2),
    Dense(units=30, activation='relu'),
    Dropout(0.1),
    Dense(units=3, activation='softmax')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 64)                320       
                                                                 
 dense_11 (Dense)            (None, 128)               8320      
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_12 (Dense)            (None, 50)                6450      
                                                                 
 dropout_7 (Dropout)         (None, 50)                0         
                                                                 
 dense_13 (Dense)            (None, 30)                1530      
                                                                 
 dropout_8 (Dropout)         (None, 30)               

In [21]:
# 3. 학습과정 설정
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [23]:
# 4.모델 학습
earlystopping = EarlyStopping(patience=40)# monitor='val_loss' 기본값
hist = model.fit(train_X, train_y, epochs=200,
                validation_split=0.1, # 학습셋의 20%는 검증셋으로
                verbose = 1, #로그출력 막음(콜백함수에서 출력)
                callbacks = [earlyStopping])

Epoch 1/200
4/4 [==============================] - 0s 18ms/step - loss: 0.0657 - accuracy: 0.9907 - val_loss: 0.0317 - val_accuracy: 1.0000
Epoch 2/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0652 - accuracy: 0.9722 - val_loss: 0.0408 - val_accuracy: 1.0000
Epoch 3/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0836 - accuracy: 0.9722 - val_loss: 0.0302 - val_accuracy: 1.0000
Epoch 4/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0846 - accuracy: 0.9722 - val_loss: 0.0203 - val_accuracy: 1.0000
Epoch 5/200
4/4 [==============================] - 0s 9ms/step - loss: 0.1008 - accuracy: 0.9722 - val_loss: 0.0213 - val_accuracy: 1.0000
Epoch 6/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0438 - accuracy: 0.9815 - val_loss: 0.0817 - val_accuracy: 0.9167
Epoch 7/200
4/4 [==============================] - 0s 10ms/step - loss: 0.0982 - accuracy: 0.9722 - val_loss: 0.0495 - val_accuracy: 1.0000
Epoch 8/200
4/4 [======

4/4 [==============================] - 0s 9ms/step - loss: 0.0722 - accuracy: 0.9815 - val_loss: 0.0211 - val_accuracy: 1.0000
Epoch 60/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0620 - accuracy: 0.9815 - val_loss: 0.0133 - val_accuracy: 1.0000
Epoch 61/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0624 - accuracy: 0.9722 - val_loss: 0.0215 - val_accuracy: 1.0000
Epoch 62/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0746 - accuracy: 0.9815 - val_loss: 0.0429 - val_accuracy: 1.0000
Epoch 63/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0680 - accuracy: 0.9815 - val_loss: 0.0313 - val_accuracy: 1.0000
Epoch 64/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0778 - accuracy: 0.9815 - val_loss: 0.0148 - val_accuracy: 1.0000
Epoch 65/200
4/4 [==============================] - 0s 9ms/step - loss: 0.0632 - accuracy: 0.9815 - val_loss: 0.0431 - val_accuracy: 1.0000
Epoch 66/200
4/4 [===============

# 2. sklearn 이용
- 원핫인코딩을 하지 않고 라벨인코딩까지만 해야 작동

In [24]:
from sklearn.neural_network import MLPClassifier

In [56]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape

((120, 4), (120,), (30, 4), (30,))

In [57]:
model = MLPClassifier(
            hidden_layer_sizes=(64, 128, 50), # hidden layer의 units 수
            activation='relu',
            solver='adam', # sgd등의 optimizer
            batch_size=40,
            max_iter=1000, # 학습 최대 횟수
            early_stopping=True, # 조기 종료 활성화
            n_iter_no_change=20, # earlyStopping의 patience와 유사
            validation_fraction=0.1, # 검증셋 비율
            warm_start=False # True일 경우 이전학습에 이어서 학습
)

In [58]:
model.fit(train_X, train_y)

MLPClassifier(batch_size=40, early_stopping=True,
              hidden_layer_sizes=(64, 128, 50), max_iter=1000,
              n_iter_no_change=20)

In [59]:
model.score(test_X, test_y)

0.9666666666666667

In [45]:
iris_X[0]

array([5.1, 3.5, 1.4, 0.2])

In [60]:
# 모델 사용
input_data = [[5.1, 3.5, 1.4, 0.2]]
model.predict(input_data)

array([0])

In [62]:
# 교차표
# test_y : 실제값
hat_y = model.predict(test_X) # 예측값
pd.crosstab(test_y, hat_y, rownames=['real'], colnames=['predict'])

predict,0,1,2
real,,,
0,10,0,0
1,0,9,1
2,0,0,10


# 3.클래스를 생성하여 모델 생성함수 사용

In [67]:
class DNNClassifier:
    @staticmethod
    def build(input_dim=4, activation='relu'):

        model = Sequential([
            Input(input_dim),
            Dense(units=50, activation=activation),
            Dense(units=30, activation=activation),
            Dense(units=3, activation='softmax')
        ])
        model.compile(loss='sparse_categorical_crossentropy',
                     optimizer='adam',
                     metrics=['accuracy'])
        return model

In [69]:
# 1. 데이터
train_X.shape, train_y.shape, test_X.shape, test_y.shape
# 2. 모델
model = DNNClassifier.build(input_dim=4, activation='elu')
# 3. 모델 학습
hist = model.fit(train_X, train_y, epochs=50, validation_split=0.1)

Epoch 1/50
4/4 [==============================] - 0s 42ms/step - loss: 1.0734 - accuracy: 0.5185 - val_loss: 0.9027 - val_accuracy: 0.8333
Epoch 2/50
4/4 [==============================] - 0s 9ms/step - loss: 0.9242 - accuracy: 0.6204 - val_loss: 0.8861 - val_accuracy: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 9ms/step - loss: 0.8412 - accuracy: 0.6759 - val_loss: 0.7551 - val_accuracy: 0.8333
Epoch 4/50
4/4 [==============================] - 0s 9ms/step - loss: 0.7519 - accuracy: 0.8796 - val_loss: 0.6728 - val_accuracy: 1.0000
Epoch 5/50
4/4 [==============================] - 0s 8ms/step - loss: 0.6872 - accuracy: 0.9537 - val_loss: 0.5770 - val_accuracy: 1.0000
Epoch 6/50
4/4 [==============================] - 0s 10ms/step - loss: 0.6294 - accuracy: 0.8611 - val_loss: 0.4957 - val_accuracy: 0.9167
Epoch 7/50
4/4 [==============================] - 0s 10ms/step - loss: 0.5845 - accuracy: 0.7593 - val_loss: 0.4444 - val_accuracy: 0.9167
Epoch 8/50
4/4 [===============

In [70]:
# 모델 평가
loss, accuracy = model.evaluate(test_X, test_y)
print(f'정확도 : {accuracy*100} %')

1/1 [==============================] - 0s 18ms/step - loss: 0.1357 - accuracy: 0.9667
정확도 : 96.66666388511658 %


# 4. 함수형 API
- 병렬처리, Resicual block

In [78]:
# 기존 model 스타일1
model = Sequential()
model.add(Input(shape=(4,)))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=30, activation='relu'))
model.add(Dense(units=3, activation='softmax'))
# 기존 model 스타일2
model = Sequential([
            Input(shape=(4,)),
            Dense(units=50, activation='relu'),
            Dense(units=30, activation='relu'),
            Dense(units=3, activation='softmax')
        ])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 50)                250       
                                                                 
 dense_29 (Dense)            (None, 30)                1530      
                                                                 
 dense_30 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [79]:
# 기존 model 스타일3 (함수형 API)
from tensorflow.keras import Model
input_ = Input(shape=(4,))
layer1 = Dense(units=50, activation='relu')(input_)
layer2 = Dense(units=30, activation='relu')(layer1)
output = Dense(units=3, activation='softmax')(layer2)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 4)]               0         
                                                                 
 dense_31 (Dense)            (None, 50)                250       
                                                                 
 dense_32 (Dense)            (None, 30)                1530      
                                                                 
 dense_33 (Dense)            (None, 3)                 93        
                                                                 
Total params: 1,873
Trainable params: 1,873
Non-trainable params: 0
_________________________________________________________________


In [80]:
# 병렬처리 방식
from tensorflow.keras.layers import concatenate
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=80, activation='relu')(input_)
dense3 = Dense(units=30, activation='relu')(input_)
x = concatenate([dense1, dense2, dense3])
output = Dense(units=3, activation='softmax')(x)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_34 (Dense)               (None, 50)           250         ['input_13[0][0]']               
                                                                                                  
 dense_35 (Dense)               (None, 80)           400         ['input_13[0][0]']               
                                                                                                  
 dense_36 (Dense)               (None, 30)           150         ['input_13[0][0]']               
                                                                                            

In [82]:
# 리지듀얼 블럭(Residual Block) : 딥러닝에서 기울기 소실 문제로 학습이 잘 되 지 않는 부분을 해결하기 위한 제안
from tensorflow.keras.layers import add, concatenate
from tensorflow.keras import Model
input_ = Input(shape=(4,))
dense1 = Dense(units=50, activation='relu')(input_)
dense2 = Dense(units=50, activation='relu')(dense1)
dense3 = add([dense1, dense2])
output = Dense(units=3, activation='softmax')(dense3)
model = Model(inputs=input_, outputs=output)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 4)]          0           []                               
                                                                                                  
 dense_41 (Dense)               (None, 50)           250         ['input_15[0][0]']               
                                                                                                  
 dense_42 (Dense)               (None, 50)           2550        ['dense_41[0][0]']               
                                                                                                  
 add_1 (Add)                    (None, 50)           0           ['dense_41[0][0]',               
                                                                  'dense_42[0][0]']         

In [83]:
# 학습과정 설정 & 학습
model.compile(loss='sparse_categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
model.fit(train_X, train_y, epochs=100, verbose=1)

Epoch 1/100
4/4 [==============================] - 0s 2ms/step - loss: 1.6143 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 2ms/step - loss: 1.3354 - accuracy: 0.1000
Epoch 3/100
4/4 [==============================] - 0s 2ms/step - loss: 1.2038 - accuracy: 0.3333
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 1.1095 - accuracy: 0.4250
Epoch 5/100
4/4 [==============================] - 0s 2ms/step - loss: 1.0251 - accuracy: 0.4167
Epoch 6/100
4/4 [==============================] - 0s 2ms/step - loss: 0.9443 - accuracy: 0.4083
Epoch 7/100
4/4 [==============================] - 0s 1ms/step - loss: 0.8703 - accuracy: 0.6833
Epoch 8/100
4/4 [==============================] - 0s 2ms/step - loss: 0.8096 - accuracy: 0.7000
Epoch 9/100
4/4 [==============================] - 0s 3ms/step - loss: 0.7576 - accuracy: 0.7250
Epoch 10/100
4/4 [==============================] - 0s 2ms/step - loss: 0.7127 - accuracy: 0.7750
Epoch 11/100
4/4 [=======

4/4 [==============================] - 0s 2ms/step - loss: 0.1086 - accuracy: 0.9833
Epoch 85/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1040 - accuracy: 0.9833
Epoch 86/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1032 - accuracy: 0.9833
Epoch 87/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1019 - accuracy: 0.9833
Epoch 88/100
4/4 [==============================] - 0s 2ms/step - loss: 0.1012 - accuracy: 0.9917
Epoch 89/100
4/4 [==============================] - 0s 2ms/step - loss: 0.0984 - accuracy: 0.9833
Epoch 90/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0977 - accuracy: 0.9833
Epoch 91/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0975 - accuracy: 0.9917
Epoch 92/100
4/4 [==============================] - 0s 2ms/step - loss: 0.0952 - accuracy: 0.9833
Epoch 93/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0946 - accuracy: 0.9833
Epoch 94/100
4/4 [===============

In [85]:
# 학습평가
loss, accuracy = model.evaluate(test_X, test_y, verbose=0)
print(f'accuracy : {accuracy}')

accuracy : 0.9666666388511658
